In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv
import numpy as np
import pandas as pd

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations, CsvParser

from common import SNOBAL_DIR, DATA_DIR, SNOTEL_DIR, COARSEN_OPTS, use_hvplot, HV_PLOT_OPTS

use_hvplot()

In [ ]:
client = start_cluster(6, 24)
client_ip_and_port(client)

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Snow Depth (m)'

## SNOTEL 

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites.json')

In [ ]:
schofield_snotel_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / 'wy2021' / f'Schofield/usda-csv/2021-Schofield-Pass.csv',
    ),
    # CsvParser.file(
    #     SNOTEL_DIR / 'wy2022' / f'Schofield/usda-csv/2022-Schofield-Pass.csv',
    # )
])

butte_snotel_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / 'wy2021'  / f'Butte/usda-csv/2021-Butte.csv',
    ),
    # CsvParser.file(
    #     SNOTEL_DIR / 'wy2022'  / f'Butte/usda-csv/2022-Butte.csv',
    # ),
])

taylor_snotel_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / 'wy2021' / f'Taylor/usda-csv/2021-Upper-Taylor.csv',
    ),
    # CsvParser.file(
    #     SNOTEL_DIR / 'wy2022' / f'Taylor/usda-csv/2022-Upper-Taylor.csv',
    # )
])

In [ ]:
irwin_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / 'wy2021' / f'Irwin/usda-csv/2021-Irwin.csv',
    ),
    #     CsvParser.file(
    #     SNOTEL_DIR / 'wy2022' / f'Irwin/usda-csv/2022-Irwin.csv',
    # )
])

irwin_csv['Depth(m)'] = irwin_csv['Depth(m)'] / 10

## Model

In [ ]:
water_year = 'wy2021'

In [ ]:
solar_HRRR = xr.open_mfdataset(
    f'{SNOBAL_DIR}/{water_year}/erw_hrrr_solar/run*/snow.nc',
    parallel=True, chunks={'time': 1, 'y' :10, 'x': 10},
)

butte_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').compute()
schofield_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').compute()
taylor_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').compute()
irwin_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

del solar_HRRR

In [ ]:
solar_HRRR = xr.open_mfdataset(
    f'{SNOBAL_DIR}/{water_year}/erw_hrrr_solar_modis/run*/snow.nc',
    parallel=True, chunks={'time': 1, 'y' :10, 'x': 10},
)

butte_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').compute()
schofield_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').compute()
taylor_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').compute()
irwin_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

del solar_HRRR

In [ ]:
solar_SMRF = xr.open_mfdataset(
    f'{SNOBAL_DIR}/{water_year}/erw/run*/snow.nc',
    parallel=True, chunks={'time': 1, 'y' :10, 'x': 10},
)

butte_snobal = solar_SMRF.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').compute()
schofield_snobal = solar_SMRF.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').compute()
taylor_snobal = solar_SMRF.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').compute()
irwin_snobal = solar_SMRF.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

# SNOTEL 

## Butte 

In [ ]:
butte_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR').opts(**HV_PLOT_OPTS) * \
butte_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR + MODIS') * \
butte_snobal.thickness.squeeze(['x', 'y']).hvplot(label='SMRF') * \
butte_snotel_csv['Depth(m)'].plot(label='SNOTEL')

## Schofield Pass 

In [ ]:
schofield_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR solar').opts(**HV_PLOT_OPTS) * \
schofield_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR + MODIS') * \
schofield_snobal.thickness.squeeze(['x', 'y']).hvplot(label='SMRF solar') * \
schofield_snotel_csv['Depth(m)'].plot(label='SNOTEL')

## Upper Taylor 

In [ ]:
taylor_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR').opts(**HV_PLOT_OPTS) * \
taylor_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR + MODIS') * \
taylor_snobal.thickness.squeeze(['x', 'y']).hvplot(label='SMRF') * \
taylor_snotel_csv['Depth(m)'].plot(label='SNOTEL')

## Irwin 

***NOTE***: Irwin guide is not a SNOTEL station and does not have a QA/QC end of day value. Using a 'rolling daily' window to plot

In [ ]:
irwin_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR').opts(**HV_PLOT_OPTS) * \
irwin_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR + MODIS') * \
irwin_snobal.thickness.squeeze(['x', 'y']).hvplot(label='SMRF') * \
irwin_csv['Depth(m)'].rolling('1D').mean().hvplot(label='SNOTEL')